In [ ]:
# Instalacion de dependencias
pip install sqlalchemy psycopg2-binary
pip install pyarrow

In [ ]:
import pandas as pd

In [ ]:
# Verificacion de pandas
pd.__version__

# Explorando Pandas

# Conectando a PostgreSQL

In [ ]:
POSTGRES_USERNAME = ""
POSTGRES_PASSWORD = ""
POSTGRES_URL = ""
POSTGRES_PORT = 5432
POSTGRES_DATABASE = ""
engine =  create_engine(f"postgresql//{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_URL}:{POSTGRES_PORT}/{POSTGRES_DATABASE}")

In [ ]:
# verificar la conexion
engine.connect()

In [ ]:
query = """
SELECT 'abc' as string;
"""
pd.read_sql(query, con=engine)

In [ ]:
# tablas presentes en el database
query = """
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
    schemaname != 'information_schema';
"""
pd.read_sql(query, con=engine)